# Semantic Kernel Tool Use Example 

## Import the Needed Packages 

In [1]:
import json
import os

from dotenv import load_dotenv

from IPython.display import display, HTML

from typing import Annotated
from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## Creating the Plugins    
Semantic Kernel uses plugins as tools that can be called by the agent. A plugin can have multiple `kernel_functions` in it as a group. 

In the example below, we create a `DestinationsPlugin` that has two functions: 
1. Provides a list of destinations using the `get_destinations` function
2. Provides a lis of availablity for each destination using the `get_availabilty` function, 

In [2]:
# Define a sample plugin for the sample
class DestinationsPlugin:
    """A List of Destinations for vacation."""

    @kernel_function(description="Provides a list of vacation destinations.")
    def get_destinations(self) -> Annotated[str, "Returns the vacation destinations."]:
        return """
        Barcelona, Spain
        Paris, France
        Berlin, Germany
        Tokyo, Japan
        New York, USA
        """

    @kernel_function(description="Provides the availability of a destination.")
    def get_availability(
        self, destination: Annotated[str, "The destination to check availability for."]
    ) -> Annotated[str, "Returns the availability of the destination."]:
        return """
        Barcelona - Unavailable
        Paris - Available
        Berlin - Available
        Tokyo - Unavailable
        New York - Available
        """
    @kernel_function(description="Provides the price of a destination.")
    def get_price(
        self, destination: Annotated[str, "The destination to check price for."]
    ) -> Annotated[str, "Returns the price of the destination."]:
        return """
        Barcelona - $2000
        Paris - $2500
        Berlin - $1800
        Tokyo - $3000
        New York - $3500
        """
    @kernel_function(description="Provides the weather of a destination.")
    def get_weather(
        self, destination: Annotated[str, "The destination to check weather for."]
    ) -> Annotated[str, "Returns the weather of the destination."]:
        return """
        Barcelona - Sunny
        Paris - Rainy
        Berlin - Cloudy
        Tokyo - Sunny
        New York - Snowy
        """
    @kernel_function(description="Provides the activities of a destination.")
    def get_activities(
        self, destination: Annotated[str, "The destination to check activities for."]
    ) -> Annotated[str, "Returns the activities of the destination."]:
        return """
        Barcelona - Beach, Sightseeing
        Paris - Museums, Shopping
        Berlin - History, Nightlife
        Tokyo - Culture, Food
        New York - Broadway, Parks
        """
    @kernel_function(description="Provides the culture of a destination.")
    def get_culture(
        self, destination: Annotated[str, "The destination to check culture for."]
    ) -> Annotated[str, "Returns the culture of the destination."]:
        return """
        Barcelona - Art, Architecture
        Paris - Fashion, Cuisine
        Berlin - History, Music
        Tokyo - Tradition, Technology
        New York - Diversity, Arts
        """
    @kernel_function(description="Provides the cuisine of a destination.")
    def get_cuisine(
        self, destination: Annotated[str, "The destination to check cuisine for."]
    ) -> Annotated[str, "Returns the cuisine of the destination."]:
        return """
        Barcelona - Tapas
        Paris - French
        Berlin - German
        Tokyo - Sushi
        New York - American
        """
    @kernel_function(description="Provides the transportation of a destination.")
    def get_transportation(
        self, destination: Annotated[str, "The destination to check transportation for."]
    ) -> Annotated[str, "Returns the transportation of the destination."]:
        return """
        Barcelona - Metro, Bus
        Paris - Metro, Bus
        Berlin - U-Bahn, S-Bahn
        Tokyo - Train, Subway
        New York - Subway, Bus
        """
    @kernel_function(description="Provides the safety of a destination.")
    def get_safety(
        self, destination: Annotated[str, "The destination to check safety for."]
    ) -> Annotated[str, "Returns the safety of the destination."]:
        return """
        Barcelona - Safe
        Paris - Safe
        Berlin - Safe
        Tokyo - Very Safe
        New York - Safe
        """
    @kernel_function(description="Provides the language of a destination.")
    def get_language(
        self, destination: Annotated[str, "The destination to check language for."]
    ) -> Annotated[str, "Returns the language of the destination."]:
        return """
        Barcelona - Spanish
        Paris - French
        Berlin - German
        Tokyo - Japanese
        New York - English
        """
    @kernel_function(description="Provides the time zone of a destination.")
    def get_time_zone(
        self, destination: Annotated[str, "The destination to check time zone for."]
    ) -> Annotated[str, "Returns the time zone of the destination."]:
        return """
        Barcelona - CET
        Paris - CET
        Berlin - CET
        Tokyo - JST
        New York - EST
        """
    @kernel_function(description="Provides the best time to visit a destination.")
    def get_best_time_to_visit(
        self, destination: Annotated[str, "The destination to check best time to visit for."]
    ) -> Annotated[str, "Returns the best time to visit the destination."]:
        return """
        Barcelona - Spring
        Paris - Spring
        Berlin - Summer
        Tokyo - Spring
        New York - Fall
        """

## Creating the Client

In this sample, we will use [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for access to the LLM. 

The `ai_model_id` is defined as `gpt-4o-mini`. Try changing the model to another model available on the GitHub Models marketplace to see the different results. 

For us to use the `Azure Inference SDK` that is used for the `base_url` for GitHub Models, we will use the `OpenAIChatCompletion` connector within Semantic Kernel. There are also other [available connectors](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) to use Semantic Kernel for other model providers.

In [3]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.getenv("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Creating the Agent 
Now we will create the Agent by using the Agent Name and Instructions that we can set. 

You can change these settings to see how the differences in the agent's response. 

In [4]:
# Create the agent
agent = ChatCompletionAgent(
    service=chat_completion_service,
    name="TravelAgent",
    instructions="Answer questions about the travel destinations, price, weather, activities, culture, cuisine, transportation, safety, language, time zone, and their availability.",
    plugins=[DestinationsPlugin()],
)

## Running the Agent 

Now we wil run the AI Agent. In this snippet, we can add two messages to the `user_input` to show how the agent responds to followup questions. 

The agent should call the correct function to get the list of available destinations and confirm the availablity of a certain location. 

You can change the `user_inputs` to see how the agent responds. 

In [5]:
user_inputs = [
    "What destinations are available?",
    "Is Barcelona available?",
    "Are there any vacation destinations available not in Europe?",
    "What is the price of Paris?",
    "What is the weather in Berlin?",
    "What activities can I do in Tokyo?",
    "What is the culture of New York?",
    "What is the cuisine of Barcelona?",
    "What is the transportation like in Paris?",
    "Is Berlin safe?",
    "What language is spoken in Tokyo?",
    "What is the time zone of New York?",
    "What is the best time to visit Barcelona?"
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()